## Rasterio with NetCDF, VSIS3, and earthaccess

In [ ]:
import numpy as np
import rasterio
import rasterio as rio
from common import earthaccess_args
from rasterio.session import AWSSession
from rasterio.warp import calculate_default_transform, reproject

In [ ]:
dataset = "gpm_imerg"
filename = earthaccess_args[dataset]["filename"]
bucket = earthaccess_args[dataset]["bucket"]
folder = earthaccess_args[dataset]["folder"]
variable = earthaccess_args[dataset]["variable"]

In [ ]:
def configure_auth():
    import boto3
    import earthaccess

    auth = earthaccess.login()
    s3_credentials = auth.get_s3_credentials("PODAAC")
    session = boto3.Session(
        aws_access_key_id=s3_credentials["accessKeyId"],
        aws_secret_access_key=s3_credentials["secretAccessKey"],
        aws_session_token=s3_credentials["sessionToken"],
        region_name="us-west-2",
    )
    rio_env = rio.Env(
        AWSSession(session),
    )
    rio_env.__enter__()
    return rio_env

In [ ]:
def warp_resample():
    input_uri = f"{folder}/{filename}"
    src = f"NETCDF:/vsis3/{bucket}/{input_uri}:{variable}"
    dstSRS = "EPSG:3857"
    srcSRS = "EPSG:4326"
    width = height = 256
    with rasterio.open(src) as da:
        dst_transform, w, h = calculate_default_transform(
            srcSRS,
            dstSRS,
            da.width,
            da.height,
            *da.bounds,
            dst_width=width,
            dst_height=height,
        )
        destination = np.zeros((width, height), np.uint8)
        return reproject(
            rasterio.band(da, 1),
            destination,
            dst_crs=dstSRS,
            dst_transform=dst_transform,
        )

In [ ]:
if __name__ == "__main__":
    rio_env = configure_auth()
    warp_resample()
    rio_env.__exit__()